In [2]:
!pip3 install alpha_vantage
!pip3 install backtrader

  Created wheel for alpha-vantage: filename=alpha_vantage-2.1.0-cp36-none-any.whl size=14345 sha256=c36d0d25fa965a53691ecd5c007c75e76cc659ed39b94de2ba102e680bfe1bfc
  Stored in directory: /root/.cache/pip/wheels/21/9c/02/b7cfae63f7119f4089f229ba1a09ce90fabbaf96199c2890be
Successfully built alpha-vantage
     |████████████████████████████████| 419kB 5.0MB/s 


In [5]:
'''
Author: www.backtest-rookies.com
 
MIT License
 
Copyright (c) 2019 backtest-rookies.com
 
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
 
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
 
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
'''


 
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np
import backtrader as bt
from datetime import datetime
 
# IMPORTANT!
# ----------
# Register for an API at:
# https://www.alphavantage.co/support/#api-key
# Then insert it here.
Apikey = 'DRZVXGEXQOQ1NAFP'
 
 
def alpha_vantage_eod(symbol_list, compact=False, debug=False, *args, **kwargs):
    '''
    Helper function to download Alpha Vantage Data.
 
    This will return a nested list with each entry containing:
        [0] pandas dataframe
        [1] the name of the feed.
    '''
    data_list = list()
 
    size = 'compact' if compact else 'full'
 
    for symbol in symbol_list:
 
        if debug:
            print('Downloading: {}, Size: {}'.format(symbol, size))
 
        # Submit our API and create a session
        alpha_ts = TimeSeries(key=Apikey, output_format='pandas')
 
        data, meta_data = alpha_ts.get_daily(symbol=symbol, outputsize=size)
 
        #Convert the index to datetime.
        data.index = pd.to_datetime(data.index)
        data.columns = ['Open', 'High', 'Low', 'Close','Volume']
 
        if debug:
            print(data)
 
        data_list.append((data, symbol))
 
    return data_list
 
class TestStrategy(bt.Strategy):
    def __init__(self):
        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=21)
        # self.sma = btind.SimpleMovingAverage(period=15)

    def next(self):
        if not self.position:
            if self.rsi < 25:
                self.buy(size=100)
                print('buy',self.data[0])
        else:
            if self.rsi > 40:
                self.sell(size=100)
                print('sell',self.data[0])

                
    def next_(self):
 
       for i, d in enumerate(self.datas):
 
            bar = len(d)
            dt = d.datetime.datetime()
            dn = d._name
            o = d.open[0]
            h = d.high[0]
            l = d.low[0]
            c = d.close[0]
            v = d.volume[0]
 
 
            print('{} Bar: {} | {} | O: {} H: {} L: {} C: {} V:{}'.format(dt, bar,dn,o,h,l,c,v))
 
 
# Create an instance of cerebro
cerebro = bt.Cerebro()
 
# Add our strategy
cerebro.addstrategy(TestStrategy)
 
# Download our data from Alpha Vantage.
symbol_list = ['VOE.VIE']
data_list = alpha_vantage_eod(
                symbol_list,
                compact=False,
                debug=False)
 
for i in range(len(data_list)):
 
    data = bt.feeds.PandasData(
                dataname=data_list[i][0], # This is the Pandas DataFrame
                name=data_list[i][1], # This is the symbol
                timeframe=bt.TimeFrame.Days,
                compression=1,
                fromdate=datetime(2019,1,1),
                todate=datetime(2019,8,28)
                )
 
    #Add the data to Cerebro
    cerebro.adddata(data)
 
print('Starting to run')
# Run the strategy
cerebro.run()

Starting to run
2019-01-31 00:00:00 Bar: 22 | VOE.VIE | O: 27.99 H: 28.2 L: 27.72 C: 27.88 V:469087.0
2019-02-01 00:00:00 Bar: 23 | VOE.VIE | O: 28.15 H: 28.15 L: 27.64 C: 27.74 V:432302.0
2019-02-04 00:00:00 Bar: 24 | VOE.VIE | O: 27.97 H: 27.97 L: 26.82 C: 26.88 V:674576.0
2019-02-05 00:00:00 Bar: 25 | VOE.VIE | O: 27.0 H: 27.55 L: 26.75 C: 26.88 V:454689.0
2019-02-06 00:00:00 Bar: 26 | VOE.VIE | O: 26.9 H: 27.4 L: 26.7 C: 27.24 V:555139.0
2019-02-07 00:00:00 Bar: 27 | VOE.VIE | O: 27.07 H: 27.07 L: 25.2 C: 25.3 V:1069870.0
2019-02-08 00:00:00 Bar: 28 | VOE.VIE | O: 25.59 H: 25.6 L: 24.64 C: 24.82 V:898583.0
2019-02-12 00:00:00 Bar: 29 | VOE.VIE | O: 25.2 H: 25.35 L: 24.92 C: 25.25 V:473332.0
2019-02-13 00:00:00 Bar: 30 | VOE.VIE | O: 25.47 H: 25.7 L: 25.25 C: 25.29 V:516903.0
2019-02-14 00:00:00 Bar: 31 | VOE.VIE | O: 25.59 H: 25.59 L: 24.94 C: 25.06 V:467885.0
2019-02-15 00:00:00 Bar: 32 | VOE.VIE | O: 25.07 H: 25.8 L: 24.81 C: 25.79 V:646148.0
2019-02-18 00:00:00 Bar: 33 | VOE.VIE